In [3]:
from collections import Counter
import numpy as np
from nltk import ngrams
import nltk.translate.bleu_score as bleu

In [4]:
# 단순 카운트 함수

def simple_count(tokens, n): # 토큰화 된 candidate 문장, n-gram에서의 n 이 두 가지를 인자로 받음.
    return Counter(ngrams(tokens, n)) #문장에서 n-gram을 카운

# count_clip 이름을 가진 함수 구현

def count_clip(candidate, reference_list, n):
    cnt_ca = simple_count(candidate, n)
    # Ca 문장에서 n-gram 카운트
    temp = dict()

    for ref in reference_list: # 다수의 Ref 문장에 대해서 이하 반복
        cnt_ref = simple_count(ref, n)
        # Ref 문장에서 n-gram 카운트

        for n_gram in cnt_ref: # 모든 Ref에 대해서 비교하여 특정 n-gram이 하나의 Ref에 가장 많이 등장한 횟수를 저장
            if n_gram in temp:
                temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) # max_ref_count
            else:
                temp[n_gram] = cnt_ref[n_gram]

    return {
        n_gram: min(cnt_ca.get(n_gram, 0), temp.get(n_gram, 0)) for n_gram in cnt_ca
        # count_clip=min(count, max_ref_count)
        # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환한다.
     }

# 보정된 정밀도를 연산하는 함수를 modified_precision란 이름의 함수로 구현

def modified_precision(candidate, reference_list, n):
    clip = count_clip(candidate, reference_list, n) 
    total_clip = sum(clip.values()) # 분자

    ct = simple_count(candidate, n)
    total_ct = sum(ct.values()) #분모

    if total_ct==0: # n-gram의 n이 커졌을 때 분모가 0이 되는 것을 방지
      total_ct=1

    return (total_clip / total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

def closest_ref_length(candidate, reference_list): # Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    return closest_ref_len

# brevity_penalty를 구현

def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1
    elif ca_len == 0 :
    # candidate가 비어있다면 BP = 0 → BLEU = 0.0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

In [5]:
def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)] 
    #p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

- candidate1 = 기존 chat gpt 
- candidate2 = 우리가 prompting 한 gpt
- references = 실제 시나리오

In [6]:
can1=[]
can2=[]

In [7]:
candidate1 = "상담의 빈도나 시간적인 제약이 있나요?"
candidate2 = "상담 방식에 대해 고민이 필요하시다면 시간을 드리겠습니다. 언제든지 편하실 때 다시 연락 주시면 상담을 이어나갈 준비가 되어 있을게요. 오늘도 좋은 하루 되시길 바랍니다."
references = [
'네. 저희가 언제든지 도와드릴 테니까 편하신 시간에 전화 주시고 방문하시면 더욱 좋습니다. 맞춤형 복지팀으로 오셔서 상담 꼭 받으세요. '
,'물론 기타 소득에 재산 관련 요건을 충족되셔야 지원 받으실 수 있으신데 혹시 관심이 있으시면 방문해 주시면 자세히 설명해 드릴 수 있어요. 이제 시간 되시면 한번 오셔도 되고 전화로도 상담 가능한데요. 시간 되세요?'
,'다름이 아니고 저희가 그런 분들을 위해서 이제 긴급 지원이나 아니면 후원금 지원하는 제도도 있으니까 저희가 한번 방문해서 좀 자세한 내용을 좀 안내해드려도 될까요?'
,'그런데 저희가 차상위 본인 부담 경감이라는 제도 신청하셔서 상담을 받고 대상자로 이제 되시면 이런 병원 약 값이라든지 진료비라든지 지금 내시는 것보다 대폭 인하된 금액으로 이용하실 수가 있거든요. 그래서 제가 봤을 때는 선생님 지금 생계라든지 의료라든지 주거라든지 이런 것들이 복합적으로 조금 어려운 부분이 있으실 수 있기 때문에 이거는 저희랑 한번 자세하게 상담을 진행을 하고 저희가 어떤 서비스로 신청해 드릴 수 있는지 안내해드리는게 좋을 것 같아요. '
,'그리고 선생님 지금은 전화 상담이기 때문에 조금 부정확할 수가 있는데 저희도 조금 알아보고 다시 추후에 연락을 드리면요 또 상담에 응하실 의향은 있으실까요?'
]

can1_score = bleu_score(candidate1.split(),list(map(lambda ref: ref.split(), references)))
can2_score = bleu_score(candidate2.split(),list(map(lambda ref: ref.split(), references)))
print("chat gpt score : " ,can1_score)
print("our gpt score : " ,can2_score)

can1.append(can1_score)
can2.append(can2_score)

chat gpt score :  0.07427357821433388
our gpt score :  0.6828267746069693


In [8]:
len(can1)

1

In [9]:
len(can2)

1

In [10]:
sum(can1) / len(can1)

0.07427357821433388

In [11]:
sum(can2) / len(can2)

0.6828267746069693